In [1]:
from reaktoro import *

In [10]:
db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase(speciate("C Na Cl"))
solution.setActivityModel(chain(ActivityModelHKF(), ActivityModelDrummond("CO2")))

gases = GaseousPhase("CO2(g) H2O(g)")
gases.setActivityModel(ActivityModelPengRobinson())

system = ChemicalSystem(db, solution, gases)

In [11]:
def mutualSolubilityCO2H2O(T, P, massNaCl):
    brine = Material(system)
    brine.add("H2O", 1.0, "kg")
    brine.add("NaCl", massNaCl, "g")

    gas = Material(system)
    gas.add("CO2(g)", 1.0, "kg")

    state = (brine + gas).equilibrate(T, "celsius", P, "bar")

    mCO2 = state.props().elementAmountInPhase("C", "AqueousPhase").val()
    xCO2 = state.props().speciesMoleFraction("H2O(g)").val() * 100.0  # mole fraction in percent unit!

    return mCO2, xCO2

In [12]:
import numpy as np

temperatures = np.linspace(0.0, 250.0)  # temperatures in °C
massesNaCl = [50.0, 100.0, 150.0]  # mass of NaCl in g

P = 100.0  # pressure in bar

table = Table()
for T in temperatures:
    table.column("Temperature") << T
    for massNaCl in massesNaCl:
        mCO2, xH2O = mutualSolubilityCO2H2O(T, P, massNaCl)
        table.column(f"mCO2:massNaCl={massNaCl}") << mCO2
        table.column(f"xH2O:massNaCl={massNaCl}") << xH2O
        
table.save("ex-mutual-solubility-co2-h2o-in-brines.txt")

In [13]:
from reaktplot import *

fig = Figure()
fig.title("CO₂ SOLUBILITY IN NACL BRINE")
for massNaCl in massesNaCl:
    fig.drawLine(table["Temperature"], table[f"mCO2:massNaCl={massNaCl}"], f"{massNaCl} g of NaCl per kg water")
fig.xaxisTitle("Temperature [°C]")
fig.yaxisTitle("CO₂ Solubility [molal]")
fig.show()

In [14]:
from reaktplot import *

fig = Figure()
fig.title("H₂O SOLUBILITY IN CO₂-RICH SUPERCRITICAL PHASE")
for massNaCl in massesNaCl:
    fig.drawLine(table["Temperature"], table[f"xH2O:massNaCl={massNaCl}"], f"{massNaCl} g of NaCl per kg water")
fig.xaxisTitle("Temperature [°C]")
fig.yaxisTitle("H₂O Solubility [%]")
fig.show()